# Download bibles from Bible.com

In [ ]:
import pandas as pd
import time

import copy
import json
import requests
from bs4 import BeautifulSoup
import os

In [ ]:
project_path = './'

struct = project_path + '00_structures/'
lang_list_fn = struct + 'languages_versions.xml'
book_list_fn = struct + 'book_list.xml'

# Output paths
csv_path = project_path + '01_input/'
tex_path = project_path + '02_outputs/'

In [ ]:
# Load info
print('Loading structural data:')

print('  - List of books in the Bible with numbers of chapters')
books = pd.read_xml(book_list_fn)

print('  - Available translations')
trans = pd.read_xml(lang_list_fn, attrs_only=True)

print('Done...')

# Show the data
#display(books)
#display(trans)

In [ ]:
def download_chapter(version, vnum, book, chapter, underscore=False):
    # Create download link
    download_link = 'https://www.bible.com/bible/' + str(vnum)+ '/' + book + '.' + str(chapter) + '.' + version
    if(underscore):
        download_link = 'https://www.bible.com/bible/' + str(vnum)+ '/' + book + '.' + str(chapter) + '_1.' + version
    # Download the text
    html_doc = requests.get(download_link)
    # Extract into Pandas df
    df = extract_verses(html_doc, version, underscore=underscore)
    return(df)

def extract_verses(html_doc, version, underscore=False, silent=False):
    soup = BeautifulSoup(html_doc.text, 'html.parser')
    # Initialize empty list to store verse data
    verse_data = []
    # Find the div element with class 'chapter'
    chapter = copy.copy(soup).find("div", {"class": "chapter"})
    book_abbr = chapter['data-usfm'].split(".")[0]
    try:
        chapter_num = int(chapter.find('div', {'class': 'label'}).text) # Most bibles use numbers for chapters
    except:
        chapter_num = chapter.find('div', {'class': 'label'}).text # Except Hebrew
    if(underscore & (~isinstance(chapter_num, int))): # Remove 'Kapitel'
        chapter_num = int(chapter_num.split(' ')[1])
    # Find the book name
    info = copy.copy(soup).find("script", {"type": "application/ld+json"}).string
    code = json.loads(info)
    book_name = code['itemListElement'][2]['item']['name'].rsplit(' ', 1)[0]
    if(underscore): # Remove 'Kapitel'
        book_name = ' '.join(book_name.split(' ')[:-1])
    # Language
    version_div = soup.find("div", {"class": "version"})
    if version_div:
        language = version_div.get("data-iso6393")
    # Status
    if(not silent):
        print('      -', book_abbr, chapter_num, '('+ book_name +')')
    # Find all verse elements
    verses = copy.copy(chapter).find_all('span', {'class': 'verse'})
    # Iterate over each verse
    for verse in verses:
        # In some cases, multiple verses are referenced as text
        verse_num = ''
        try:
            verse_num = int(copy.copy(verse).find('span', {'class': 'label'}).text)
        except:
            pass
        if(verse_num is None):
            verse_num = copy.copy(verse).find('span', {'class': 'label'}).text
        # Replace the divine name with capitals
        try:
            # English uses class nd
            divineName = verse.find('span', {'class':'nd'}).string
            verse.find('span', {'class':'nd'}).string.replace_with('\\textsc{' + divineName + '}')
        except:
            pass
        try:
            # underscore uses class sc
            divineName = verse.find('span', {'class':'sc'}).string
            verse.find('span', {'class':'sc'}).string.replace_with('\\textsc{' + divineName + '}')
        except:
            pass
        # Keep only relevant text data
        verse_text = ''
        for match in verse.find_all('span', {'class':'content'}):
            verse_text += match.get_text()
        verse_text = verse_text.strip()
        # Append verse data to list
        verse_data.append({'lang': language,
                           'version': version,
                           'book_name': book_name,
                           'book_abbr': book_abbr,
                           'chapter_num': chapter_num,
                           'verse_num': verse_num,
                           'verse_text': verse_text})
    # Create a pandas dataframe from the list of verse data
    df = pd.DataFrame(verse_data)
    # Remove extra line breaks etc
    df = df.groupby(['lang','version','book_name','book_abbr','chapter_num','verse_num'])['verse_text'].apply(lambda x: ' '.join(x)).reset_index()
    return(df)

def download_book(current_version, current_book):
    current_lang = trans.loc[trans['version'] == current_version, 'lang'].values[0]
    current_vnum = trans.loc[trans['version'] == current_version, 'vnum'].values[0]
    current_lcode = trans.loc[trans['version'] == current_version, 'lcode'].values[0]
    current_order = books.loc[books['book'] == current_book, 'order'].values[0]
    current_nb_chaps = books.loc[books['book'] == current_book, 'chapters'].values[0]
    print('  - ' + current_lang + ' - ' + current_version)
    print('    - ' + current_book)
    # Some (German) bibles use a different format for the download link
    if(current_version in ['SCHL2000', 'NGU2011']):
        underscore = True
    else:
        underscore = False
    chapter_df = []
    for chap in range(1,current_nb_chaps+1):
        temp = download_chapter(current_version, current_vnum, current_book, chap, underscore=underscore)
        chapter_df.append(temp)
        time.sleep(5) # This website will not let us download quickly (protection mechanism). Pause between downloads...
        pass
    print('      - Concatenate book df...')
    book_df = pd.concat(chapter_df)
    print('      - Save to csv file: ', end = '')
    book_csv_fn = '{:02d}'.format(current_order) + '_' + current_book + '.csv'
    folder_name = current_lcode + '_' + current_version
    print(csv_path + folder_name + '/' + book_csv_fn)
    if not os.path.exists(csv_path + '/' + folder_name):
        os.makedirs(csv_path + '/' + folder_name)
    book_df.to_csv(csv_path + '/' + folder_name + '/' + book_csv_fn, index=False)
    print('Done...')
    pass

def download_version(current_version):
    print('Downloading:')
    # Cycle through and download
    for index, book in books.iterrows():
        download_book(current_version, book['book'])
    pass

def download_all():
    # Cycle through and download
    for index, tr in trans.iterrows():
        download_version(tr['version'])
    pass

In [ ]:
# Download everything
#download_all()

In [ ]:
# Download a specific version
download_version('ASV')

In [ ]:
# Download a specific book
#download_book('ASV', 'PSA')